In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy
!pip install omegaconf
!pip install gradio

  Using cached gradio-5.12.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.5.4-py3-none-any.whl.metadata (7.1 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.9.2-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.45.2-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
import random
import torch
import os
from PIL import Image
from diffusers import DiffusionPipeline, StableDiffusionPipeline, StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, UniPCMultistepScheduler
from omegaconf import OmegaConf
import mediapy as media
import sys

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def load_model(model_choice):
    if model_choice == 'sdxl':
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            use_safetensors=True,
            variant="fp16"
        )
        pipe = pipe.to("cuda")
        use_refiner = False
        refiner = None
    elif model_choice == 'sdxl + refiner':
        pipe = DiffusionPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            use_safetensors=True,
            variant="fp16"
        )
        refiner = DiffusionPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-refiner-1.0",
            text_encoder_2=pipe.text_encoder_2,
            vae=pipe.vae,
            torch_dtype=torch.float16,
            use_safetensors=True,
            variant="fp16",
        )
        refiner = refiner.to("cuda")
        pipe.enable_model_cpu_offload()
        use_refiner = True
    else:
        raise ValueError("Invalid model choice")
    return pipe, use_refiner, refiner

model_choice = "sdxl"
pipe, use_refiner, refiner = load_model(model_choice)

def generate_images(prompt, negative_prompt, image_count, style, seed, random_seed, width, height, sampler, steps):
    if random_seed:
        seed_final = random.randint(0, sys.maxsize)
    else:
        seed_final = seed

    if style != 'No Style':
        prompt_styled = prompt + ' in ' + style + ' style'
    else:
        prompt_styled = prompt

    if sampler == "Euler A":
        solver = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sampler == "DPM++":
        solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    elif sampler == "DPM++ Karras":
        solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        solver.use_karras_sigmas = True
    elif sampler == "UniPC":
        solver = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.scheduler = solver

    output_folder = "output_images"
    os.makedirs(output_folder, exist_ok=True)

    generated_images = []
    for i in range(image_count):
        images = pipe(
            prompt=prompt_styled,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            num_inference_steps=steps,
            output_type="latent" if use_refiner else "pil",
            generator=torch.Generator("cuda").manual_seed(seed_final + i)
        ).images

        if use_refiner:
            images = refiner(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=images
            ).images

        for j, image in enumerate(images):
            image_path = os.path.join(output_folder, f"{i+seed_final}_{prompt_styled[:10]}.jpg")
            image.save(image_path)
            generated_images.append(image)

    return generated_images

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
styles = ["Vintage",
          "Old Rustic",
          "Futursistic",
          "Scandinavian",
          "Coastal",
          "Traditional",
          "Isometric",
          "3D Model",
          "Minamalist",
          "Classic"]

samplers = ["Euler A", "DPM++", "DPM++ Karras", "UniPC"]

def gradio_interface():
    gr.Interface(
        fn=generate_images,
        inputs=[
            gr.Textbox(label="Prompt", value="a cozy room with warm colors, comfortable seating, ornate rugs, and wooden furniture. Add elements like a fireplace with a mantel, decorative molding, and vintage-inspired lighting fixtures to create a timeless and elegant atmosphere"),
            gr.Textbox(label="Negative Prompt", value="duplicate, ugly, horror, multiple limbs, bad anatomy, bad-artist, blurry, deformed, fake, aniamted"),
            gr.Slider(label="Image Count", minimum=1, maximum=100, step=1, value=2),
            gr.Dropdown(label="Style", choices=styles, value="No Style"),
            gr.Number(label="Seed", value=1),
            gr.Checkbox(label="Random Seed"),
            gr.Slider(label="Width", minimum=8, maximum=2048, step=8, value=512),
            gr.Slider(label="Height", minimum=8, maximum=2048, step=8, value=512),
            gr.Dropdown(label="Sampler", choices=samplers, value="DPM++ Karras"),
            gr.Slider(label="Steps", minimum=1, maximum=200, step=1, value=20),
        ],
        outputs=[
            gr.Gallery(label="Generated Images")
        ],
        title="DreamDeco",
        description="Generate interior and architectural design ideas",
        theme = gr.themes.Monochrome()
    ).launch(debug=True)

gradio_interface()

/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: No Style or set allow_custom_value=True.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://48310337a9801607fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]